In [ ]:
# requires an API key from airnow.gov

# how to get an API Key:
https://docs.airnowapi.org/faq

# api fact sheet
https://docs.airnowapi.org/docs/AirNowAPIFactSheet.pdf

# RSS data feeds
https://docs.airnowapi.org/feeds

# file products (after logging in)
https://docs.airnowapi.org/files

In [74]:
import requests

# define your api key
api_key=''

In [30]:
def curr_obs(zip_code,api_key,file_format='text/csv',dist='25'):
  # current conditions

  params={'format':file_format,'zipCode':zip_code,'distance':dist,'API_KEY':api_key}
  my_url='https://www.airnowapi.org/aq/observation/zipCode/current/'

  response = requests.get(my_url,params)
  return(response.content)

In [63]:
def forecast(zip_code,obs_date,api_key,file_format='text/csv',dist='25'):
  # obs_date: YYYY-MM-DD
  # returns text/csv, by default

  params={'format':file_format,'zipCode':zip_code,'date':obs_date,'distance':dist,'API_KEY':api_key}
  my_url='https://www.airnowapi.org/aq/forecast/zipCode/'

  response = requests.get(my_url,params)
  return(response.content)

In [66]:
def hist_obs(zip_code,obs_date,api_key,file_format='text/csv',dist='25'):
  # date format: 'YYYY-MM-DD'

  obs_date=obs_date+'T00-0000'

  params={'format':file_format,'zipCode':zip_code,'date':obs_date,'distance':dist,'API_KEY':api_key}
  my_url='https://www.airnowapi.org/aq/observation/zipCode/historical/'

  response = requests.get(my_url,params)
  return(response.content)

In [75]:
# get forecast for pittsburgh (15213) and return json format
forecast('15213','2023-07-04',file_format='application/json',api_key=my_key)

b'[{"DateIssue":"2023-07-04 ","DateForecast":"2023-07-04 ","ReportingArea":"Pittsburgh","StateCode":"PA","Latitude":40.434,"Longitude":-79.984,"ParameterName":"O3","AQI":60,"Category":{"Number":2,"Name":"Moderate"},"ActionDay":false,"Discussion":"Current Conditions: At 3 p.m. EDT this Tuesday afternoon, skies were partly sunny across the region with temperatures moving into the middle 80s. For now, PM2.5 is mainly in the good range, with ozone rising to mid to high moderate concentrations with the aid of sunshine.  Tonight into Wednesday morning, PM2.5 will increase due to smoke generated from fireworks displays, and this will end up being trapped later due to a moderately strong inversion.  ***  Wednesday\xe2\x80\x99s forecast: An Air Quality Action Day will be declared Wednesday in regard to ozone development. Residual smoke with partly to mostly sunny skies and temperatures approaching the 90-degree mark in addition to weekday emissions and light winds will contribute to ozone level

In [79]:
import pandas as pd
from io import BytesIO,StringIO

forecast_df=pd.read_csv(BytesIO(forecast('15213','2023-07-08',api_key=my_key)))
forecast_df

,DateIssue,DateForecast,ReportingArea,StateCode,Latitude,Longitude,ParameterName,AQI,CategoryNumber,CategoryName,ActionDay,Discussion
0,2023-07-06,2023-07-08,Pittsburgh,PA,40.434,-79.984,O3,70,2,Moderate,False,Current Conditions: At 3 p.m. EDT this Thursda...
1,2023-07-06,2023-07-08,Pittsburgh,PA,40.434,-79.984,PM2.5,58,2,Moderate,False,Current Conditions: At 3 p.m. EDT this Thursda...


In [67]:
current_df=pd.read_csv(BytesIO(curr_obs('60201',api_key=my_key)))
current_df.T

,0,1
DateObserved,2023-07-06,2023-07-06
HourObserved,20,20
LocalTimeZone,CST,CST
ReportingArea,Chicago,Chicago
StateCode,IL,IL
Latitude,41.964,41.964
Longitude,-87.659,-87.659
ParameterName,O3,PM2.5
AQI,38,74
CategoryNumber,1,2


In [82]:
# get historical data for pittsburgh (15213) and return xml object
hist_obs(zip_code='15213',obs_date='2023-07-04',file_format='application/xml',api_key=my_key)

b'<ObsByZipList>\n  <ObsByZip>\n    <DateObserved>07/04/2023 12:00:00 AM</DateObserved>\n    <HourObserved>0</HourObserved>\n    <LocalTimeZone>EST</LocalTimeZone>\n    <ReportingArea>Pittsburgh</ReportingArea>\n    <StateCode>PA</StateCode>\n    <Latitude>40.434</Latitude>\n    <Longitude>-79.984</Longitude>\n    <ParameterName>OZONE</ParameterName>\n    <AQI>47</AQI>\n    <CategoryNumber>1</CategoryNumber>\n    <CategoryName>Good</CategoryName>\n  </ObsByZip>\n  <ObsByZip>\n    <DateObserved>07/04/2023 12:00:00 AM</DateObserved>\n    <HourObserved>0</HourObserved>\n    <LocalTimeZone>EST</LocalTimeZone>\n    <ReportingArea>Pittsburgh</ReportingArea>\n    <StateCode>PA</StateCode>\n    <Latitude>40.434</Latitude>\n    <Longitude>-79.984</Longitude>\n    <ParameterName>PM2.5</ParameterName>\n    <AQI>69</AQI>\n    <CategoryNumber>2</CategoryNumber>\n    <CategoryName>Moderate</CategoryName>\n  </ObsByZip>\n</ObsByZipList>'

In [83]:
hist_df=pd.read_csv(BytesIO(hist_obs(zip_code='15213',obs_date='2023-07-04',api_key=my_key)))
hist_df.T

,0,1
DateObserved,2023-07-04,2023-07-04
HourObserved,0,0
LocalTimeZone,EST,EST
ReportingArea,Pittsburgh,Pittsburgh
StateCode,PA,PA
Latitude,40.434,40.434
Longitude,-79.984,-79.984
ParameterName,OZONE,PM2.5
AQI,47,69
CategoryNumber,1,2
